In [2]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

# Load your data
df = pd.read_csv('antibacterial.csv')

# Convert 'Sold_date' to datetime format
df['Sold_date'] = pd.to_datetime(df['Sold_date'], format='%m/%d/%y')

# Set 'Sold_date' as the index
df.set_index('Sold_date', inplace=True)

# Aggregate sales data on a weekly basis for each product
weekly_sales = df.groupby([pd.Grouper(freq='W'), 
'Product_details']).agg(total_sold=('Sold_quantity', 'sum')).reset_index()

# Apply one-hot encoding to 'Product_details'
weekly_sales_encoded = pd.concat([weekly_sales, pd.get_dummies(weekly_sales['Product_details'], 
prefix='product')], axis=1).drop('Product_details', axis=1)

# Extract year and week number from 'Sold_date' for temporal features
weekly_sales_encoded['year'] = weekly_sales_encoded['Sold_date'].dt.year
weekly_sales_encoded['week_of_year'] = weekly_sales_encoded['Sold_date'].dt.isocalendar().week

# Normalize the temporal features 
scaler = MinMaxScaler()
features_columns = ['year', 'week_of_year'] + [col for col in weekly_sales_encoded.columns 
if col.startswith('product')]
weekly_sales_encoded[features_columns] = scaler.fit_transform(weekly_sales_encoded[features_columns])

# Split the data into features and target
X = weekly_sales_encoded[features_columns].values
y = weekly_sales_encoded['total_sold'].values

# Adjusting the dataset split into 70% training, 15% validation, and 15% testing
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42)  # 0.1765 of 85% is roughly 15% of the whole

# Define the model architecture
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(1, activation='relu')    # Output layer
])


# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error')

# Train the model with the validation data
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=10)

# Evaluate the model
predictions = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'RMSE on test set: {rmse}')

# Prepare forecasting features for Feb 1 to 8, 2024, correctly this time
forecast_results = []
for product_col in [col for col in features_columns if col.startswith('product')]:
    forecast_period_features = np.zeros((1, len(features_columns)))
    # Find the index of the year, week_of_year, and current product column
    year_index = features_columns.index('year')
    week_of_year_index = features_columns.index('week_of_year')
    product_index = features_columns.index(product_col)
    
    # Normalize year and week_of_year values
    forecast_period_features[0, year_index] = (2024 - scaler.data_min_[year_index]) / (scaler.data_max_[year_index] - scaler.data_min_[year_index])
    forecast_period_features[0, week_of_year_index] = (5 - scaler.data_min_[week_of_year_index]) / (scaler.data_max_[week_of_year_index] - scaler.data_min_[week_of_year_index])
    forecast_period_features[0, product_index] = 1  # Activate current product

    # Predict and store the result
    predicted_quantity = model.predict(forecast_period_features).flatten()[0]
    product_name = product_col.replace('product_', '')  # Assuming prefix 'product_' is used
    forecast_results.append((product_name, predicted_quantity))

# Display forecasted quantities for each product in a DataFrame
forecast_df = pd.DataFrame(forecast_results, columns=['Product', 'Forecasted Quantity']).sort_values(by='Forecasted Quantity', ascending=False)
print(forecast_df)

Epoch 1/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 860us/step - loss: 222.3330 - val_loss: 189.4497
Epoch 2/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step - loss: 149.1417 - val_loss: 133.6473
Epoch 3/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 442us/step - loss: 117.9239 - val_loss: 128.7063
Epoch 4/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 466us/step - loss: 110.2770 - val_loss: 132.9160
Epoch 5/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 442us/step - loss: 99.9440 - val_loss: 127.6022
Epoch 6/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 434us/step - loss: 104.5966 - val_loss: 128.0410
Epoch 7/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 472us/step - loss: 119.0463 - val_loss: 124.9413
Epoch 8/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 441us/step - loss: 118.1453 - val_loss: 126.5774
Epoch 9/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 475us/step - loss: 96.3797 - val_loss: 125.5922
Epoch 10/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 438us/step - loss: 96.9870 - val_loss: 125.5521
Epoch 11/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 438us/step - loss: 101.6114

105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 554us/step - loss: 104.6117 - val_loss: 106.0779
Epoch 66/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 488us/step - loss: 93.3113 - val_loss: 107.9387
Epoch 67/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step - loss: 79.0988 - val_loss: 111.3105
Epoch 68/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 485us/step - loss: 83.7461 - val_loss: 105.9481
Epoch 69/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 530us/step - loss: 91.4665 - val_loss: 103.3874
Epoch 70/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 500us/step - loss: 73.0536 - val_loss: 110.7682
Epoch 71/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step - loss: 96.7435 - val_loss: 103.6002
Epoch 72/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 519us/step - loss: 86.5916 - val_loss: 104.7773
Epoch 73/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 517us/step - loss: 121.4389 - val_loss: 103.2338
Epoch 74/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 527us/step - loss: 87.2331 - val_loss: 104.0405
Epoch 75/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 594us/step - loss: 129.3076 - val_lo

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step